In [27]:
import re
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [28]:
train = pd.read_csv('./data/train.csv')
train = train[['selected_text','sentiment']]
train["selected_text"].fillna("No content", inplace = True)

In [29]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [30]:
temp = []
#Splitting pd.Series to list
data_to_list = train['selected_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))

In [31]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

In [32]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [33]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [34]:
data = np.array(data)

In [35]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [36]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

In [37]:
import keras.models
best_model = keras.models.load_model("Sentiment_analysis_BiLSTM.hdf5")

In [55]:
sentiment = [0,-1,1]

In [56]:
test=pd.read_csv("Messages.csv")
test.head()

,message
0,Hello bro\n
1,Hello how do you do\n
2,Yeah find\n
3,Fine... My bad\n
4,As a daily wage worker how much do you earn.\n


In [57]:
sent=[]
for i in test['message']:
    sequence = tokenizer.texts_to_sequences([i])
    sentest = pad_sequences(sequence, maxlen=200)
    sent.append(sentiment[np.around(best_model.predict(sentest), decimals=0).argmax(axis=1)[0]])

In [60]:
test['Sentiment']=sent
test

,message,Sentiment
0,Hello bro\n,0
1,Hello how do you do\n,0
2,Yeah find\n,0
3,Fine... My bad\n,0
4,As a daily wage worker how much do you earn.\n,0
...,...,...
84,In every match there will be a game changee\n,0
85,I think you should be serious with your life\n,0
86,It's been days since I fucked my gf\n,0
87,Great greater greatest\n,1


In [59]:
test.to_csv("after sent.csv")

PermissionError: [Errno 13] Permission denied: 'after sent.csv'

In [ ]:
test.head(70)